In [2]:
# import modules
from bs4 import BeautifulSoup
import requests
import csv
import re
from urllib.request import urlopen
import urllib.parse
import pandas as pd

**Scrape table:** extracts are links that redirects to the individual listings.

**Date and time of scraping:** 13.04.2024 1:00 am

In [1]:
def scrape_table():
  # Variables for page and maximum number of pages
  page = 0
  max_pages = 94

  # List for listings from table
  listing_list = []

  # Loop through all 94 pages
  for x in range(max_pages):
      print('Scraping page ' + str(page) + '.')

      # Getting the html
      url = "https://www.daft.ie/sharing/ireland?sort=publishDateDesc&pageSize=20&from=" + str(page)
      html = requests.get(url)
      soup = BeautifulSoup(html.text, "lxml")

      # Find all links that redirects to each renting listing
      listings = soup.find_all('a', href=True)
      for listing in listings:
          if 'share' in str(listing['href']):  # Extract only links that lead to rent listing
              listing_list.append(listing['href'])  # Append the link to the list

      # Setting up page number +20
      page += 20

  listing_list = set(listing_list)  # Drop duplicates
  for listing in listing_list:    # Print the result
      print(listing)
  print(len(listing_list))

  # creates text file with all href links
  with open("daft.ie.txt",encoding='utf-8', mode="w+") as file_object:
      # Append href at the end of file
      file_object.write('\n'.join(listing_list))
scrape_table()

Scraping page 0.
Scraping page 20.
Scraping page 40.
Scraping page 60.
Scraping page 80.
Scraping page 100.
Scraping page 120.
Scraping page 140.
Scraping page 160.
Scraping page 180.
Scraping page 200.
Scraping page 220.
Scraping page 240.
Scraping page 260.
Scraping page 280.
Scraping page 300.
Scraping page 320.
Scraping page 340.
Scraping page 360.
Scraping page 380.
Scraping page 400.
Scraping page 420.
Scraping page 440.
Scraping page 460.
Scraping page 480.
Scraping page 500.
Scraping page 520.
Scraping page 540.
Scraping page 560.
Scraping page 580.
Scraping page 600.
Scraping page 620.
Scraping page 640.
Scraping page 660.
Scraping page 680.
Scraping page 700.
Scraping page 720.
Scraping page 740.
Scraping page 760.
Scraping page 780.
Scraping page 800.
Scraping page 820.
Scraping page 840.
Scraping page 860.
Scraping page 880.
Scraping page 900.
Scraping page 920.
Scraping page 940.
Scraping page 960.
Scraping page 980.
Scraping page 1000.
Scraping page 1020.
Scraping page 10

In [3]:
def find_by_label(container, label):
        return container.find("span", text=re.compile(label)).next_sibling.next_sibling.next_sibling
        

In [4]:
daft_url = "https://www.daft.ie"
information = []

with open('daft.ie.txt', 'r') as f:    # open txt.file to access each link
      
    # get html for each link 
    for url in f:
        try:
            
            urls = daft_url+url
            uClient = urlopen(urls)
            page_html = uClient.read()
            uClient.close()
            soup = BeautifulSoup(page_html, "lxml")
            container = soup.find("div", {"class":"styles__MainColumn-sc-15fxapi-1"})
            #print(container)

            # find information
        
            info = dict()
        
            info["Adress"] = container.find("h1", {"class":"TitleBlock__Address-sc-1avkvav-8 dzihxK"}).text.strip()
            info["Price"] = container.find("h2", {"class": "TitleBlock__StyledCustomHeading-sc-1avkvav-5"}).text.strip()
            info["Room"] = container.find("p", {"data-testid":"beds"}).text.strip()
            info["Bath"] = container.find("p",{"data-testid": "baths"}).text.strip()
            info["Property_type"] = container.find("p", {"data-testid":"property-type"}).text.strip()
        
            ## property overview
            info["Bedrooms_available"] = find_by_label(container, "Bedrooms Available").strip()
            info["Available_from"] = find_by_label(container, "Available From").strip()
            info["Available_for"] = find_by_label(container, "Available For").strip()
            info["Sharing_with"] = find_by_label(container, "Sharing with").strip()
            info["Owner_occupied"] = find_by_label(container, "Owner Occupied").strip()
            info["Preferences"] = find_by_label(container, "Preferences").strip()
        
            ## Description
            info["Description"] = container.find("div", {"data-testid":"description"}).text.strip()
        
            ## Statistics 
            info["Date_entered/renewed"] = container.find_all("p", {"class": "Statistics__StyledLabel-sc-15tgae4-1 iDjRee"})[0].text.strip()
            info["Property_views"] = container.find_all("p", {"class", "Statistics__StyledLabel-sc-15tgae4-1 iDjRee"})[1].text.strip()
            information.append(info)
            
        except:
            pass

# write csv

filename = 'daft_listings.csv'
with open(filename, 'w', newline='') as m:
    w = csv.DictWriter(m, ["Adress","Price","Room","Bath","Property_type","Bedrooms_available","Available_from", "Available_for", "Sharing_with", "Owner_occupied",
                          "Preferences", "Description", "Date_entered/renewed", "Property_views"])
    w.writeheader()
    for info in information:
        w.writerow(info)